## Bloom Filters

### The Problem:
With **bloom filters**, the aim is to solve the **belonging problem**: we want to check whether or not an element is part of a set.
With big sets of data, linear search (storing a list of elements and then looking through it) can be reeaaaaly slow - and exceed memory!

The way this solution works is, when we only need to know if a element belongs to the set ("YES"/"NO" answer), we can leave out the storing of the elements and start only storing binary information   

### Bloom Filters:
Bloom filters use **hash functions** to calculate a vector of bits that represents the set. The element is "stored" in the vector by setting all the values returned by the hash functions to 1. The belonging in the set is determined by the contents of the vector in the positions returned by the hash functions applied to the element we're searching. 

In [ ]:
# Run if mmh3 isn't installed on your computer

import sys
!{sys.executable} -m pip install mmh3

In [15]:
import numpy
import mmh3
import pandas as pd

### Hash Functions:
It is important to choose our hash functions wisely. First of all, it has to select all elements of the vector with the same probability, so **they must be uniformely distributed**. Then, it's important that our hash functions are independent among them, otherwise it would be the same as just using **one** hash function. 

In this work we used the murmur function with different seeds, mainly because it fills the criteria and because it's fast.

### Optimal Number of Hash Functions:

$ optimal_k = (0.693*n)/m $

This limits the lower limit for the error probability: $ optimal_p = 2^{(-optimal_k)} $

In [16]:
class BloomFilter(object):
    def __init__(self, nitems, false_positive_prob):
        self.size = int(-(nitems * numpy.log(false_positive_prob))/(numpy.log(2)**2))+1 # size of the bloom filter
        self.nhash = int((0.639*self.size)/nitems)+1 # number of hash functions

        self.bloomf = numpy.zeros(self.size) # initializing the bloom filter as an array of size self.size with everything at 0
 
    def add(self, item):
        for i in range(self.nhash):
            h = mmh3.hash(item, i) % self.size #i = seed
            self.bloomf[h] = 1
 
    def check(self, item):
        for i in range(self.nhash):

            h = mmh3.hash(item, i) % self.size

            if self.bloomf[h] == 0:
                return False

        return True

In [17]:
chunk = pd.read_csv("dataset/dset.csv", chunksize= 1000000)
our_set = pd.concat(chunk)

In [18]:
our_set.head()

,Unnamed: 0,Title,Author,Subjects
0,0,A tale of two friends,"O'Ryan, Ellie","Musicians Fiction, Bullfighters Fiction, Best ..."
1,1,"Naruto. Vol. 1, Uzumaki Naruto","Kishimoto, Masashi","Ninja Japan Comic books strips etc, Comic book..."
2,2,"Peace, love & Wi-Fi : a ZITS treasury","Scott, Jerry",Duncan Jeremy Fictitious character Comic books...
3,3,The Paris pilgrims : a novel,"Carlile, Clancy","Hemingway Ernest 1899 1961 Fiction, Biographic..."
4,4,"Erotic by nature : a celebration of life, of l...",NaN,"Erotic literature American, American literatur..."


### False Results:
Being a probabilistic approach, bloom filters allow for **false positives**, values that are considered as being in the set when in reality they don't. This is caused by  "**collisions**", when two different symbols produce the same hash code. A way of minimizing this is by using multiple hash functions, at expenses of more memory usage.


The false positive rate is diminished by more hash functions only until an optimal value is reached. After that, the rate rises again because there start to be too many 1s for the same element. 


On the other hand, bloom filters don't allow for **false negatives**: if a position in the vector (or one of them) wasn't set as 1, then we can be sure that that element wasn't mapped into the vector.


### Probability of False Positives:
Being m the set dimension, n the number of positions in the vector, and k the number of hash functions,


the probability of false positives can be given by:
$ p(m,n,k) = (1-e^{(-k*m)/n})^k $ 

In [19]:
n = len(our_set) 
p = 0.001 #false positive probability

titles_bf = BloomFilter(n,p)
print(titles_bf.nhash)
print(titles_bf.size)

authors_bf = BloomFilter(n,p)
subjects_bf = BloomFilter(n,p)

10
14428183


In [20]:
file_object = open('subjects.txt', 'a')
for _,item in our_set.iterrows():
    title = item["Title"]
    author = item["Author"]
    subjects = item["Subjects"]

    if title and isinstance(title, str):
        titles_bf.add(title)

    if author and isinstance(author, str):
        authors_bf.add(author)

    if subjects and isinstance(subjects, str):
        sub_list = str(subjects).split(", ")
        for subject in sub_list:
            subjects_bf.add(subject)
file_object.close()

### Use Case
Searching if in the Seattle Public Library there is a certain book, there is a some book from a certain author or if there is a book about a certain subject. 

For this purpose, you can search by Title, Author or Subject.

In [21]:
def print_menu():
    print(30 * "-" , "WELCOME TO OUR LIBRARY" , 30 * "-")
    print("1. Search by Title")
    print("2. Search by Author")
    print("3. Search by Subject")
    print("0. Exit")
    print(84 * "-")

print_menu()

def main():
    loop = True

    while loop:
        try:
            choice = int(input("Enter your choice [1-3] or 0 to quit: "))
        except:
            choice = 222
        
        if choice==1:     
            inpt = input("Enter the title: ")
            if titles_bf.check(inpt):
                print("The searched book is in our library!")
            else:
                print("We're sorry, but we didn't find a result.")
        elif choice==2:     
            inpt = input("Enter the author: ")

            # if name is introduced "raw" (first + space + last), we transform it into last + comma + space + first
            if len(inpt.split(",")) == 1 and len(inpt.split(" ")) > 1: # 2 names but no comma separation
                inpt = ", ".join(inpt.split(" ")[::-1])
            
            if authors_bf.check(inpt):
                print("We have books of the searched author in our library!")
            else:
                print("We're sorry, but we didn't find a result.")
        elif choice==3:     
            inpt = input("Enter the subject: ")
            if subjects_bf.check(inpt):
                print("We have books about the searched subject in our library!")
            else:
                print("We're sorry, but we didn't find a result.")
        elif choice==0:
            print("See you soon!")
            break
        else:
            # Any inputs other than values 0-3 we print an error message
            print("Wrong option selected.")

main()

------------------------------ WELCOME TO OUR LIBRARY ------------------------------
1. Search by Title
2. Search by Author
3. Search by Subject
0. Exit
------------------------------------------------------------------------------------
See you soon!


### Some Conclusions:
#### Limitations:
* We need to know the maximum number of elements to store in the vector beforehand
* The number of false positives can rapidly increase, but this can be traded off with the number of hash functions and space to store the vector 

#### Upsides:
* It's a really fast method
* It has a relatively low error rate
* It works for a big dataset